# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846688862671                   -0.70    4.5         
  2   -7.852255266276       -2.25       -1.53    1.0   93.7ms
  3   -7.852612306857       -3.45       -2.56    1.8   29.7ms
  4   -7.852645982088       -4.47       -2.89    2.5   36.0ms
  5   -7.852646514570       -6.27       -3.20    1.0   27.3ms
  6   -7.852646679196       -6.78       -4.06    1.0   27.7ms
  7   -7.852646686587       -8.13       -5.17    2.0   35.3ms
  8   -7.852646686726       -9.86       -5.57    1.5   30.6ms
  9   -7.852646686727      -12.06       -5.55    1.5   30.7ms
 10   -7.852646686730      -11.54       -6.72    1.0   27.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846693141151                   -0.70           4.5         
  2   -7.852549692793       -2.23       -1.64   0.80    2.2    282ms
  3   -7.852637870922       -4.05       -2.74   0.80    1.0   25.2ms
  4   -7.852646430496       -5.07       -3.32   0.80    2.0   31.5ms
  5   -7.852646676988       -6.61       -4.15   0.80    2.0   30.7ms
  6   -7.852646686393       -8.03       -4.75   0.80    1.5   91.5ms
  7   -7.852646686724       -9.48       -5.78   0.80    1.8   29.0ms
  8   -7.852646686730      -11.25       -6.75   0.80    2.2   31.7ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.415417e+01     3.878634e+00
 * time: 0.0639040470123291
     1     1.357454e+00     2.232629e+00
 * time: 0.38184094429016113
     2    -1.409403e+00     2.713847e+00
 * time: 0.4078691005706787
     3    -3.711272e+00     2.354088e+00
 * time: 0.44581103324890137
     4    -4.909513e+00     2.121931e+00
 * time: 0.4833240509033203
     5    -6.667353e+00     1.155858e+00
 * time: 0.5208511352539062
     6    -7.394385e+00     3.783053e-01
 * time: 0.5582461357116699
     7    -7.628805e+00     1.843477e-01
 * time: 0.5848960876464844
     8    -7.702457e+00     1.085585e-01
 * time: 0.6109521389007568
     9    -7.736438e+00     8.232195e-02
 * time: 0.6373059749603271
    10    -7.761303e+00     6.825428e-02
 * time: 0.6640670299530029
    11    -7.783244e+00     7.692851e-02
 * time: 0.6909940242767334
    12    -7.803324e+00     9.472467e-02
 * time: 0.7177960872650146
    13    -7.826983e+00     6.808226e-02
 * time: 0.74404907

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846899854742                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645935716                   -1.64         
  2   -7.852646686730       -6.12       -3.71    2.13s
  3   -7.852646686730      -13.35       -7.27    165ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.092403935897309e-7
|ρ_newton - ρ_scfv| = 2.1568428993264149e-7
|ρ_newton - ρ_dm|   = 9.088936397427687e-10
